In [ ]:
import os
MODIS_dir="/content/drive/MyDrive/Data_county_pull_modis_entire_country"

In [ ]:
!pip install 2to3
!pip install absl-py
!pip install appnope
!pip install asn1crypto
!pip install astor
!pip install backcall
!pip install beautifulsoup4
!pip install bleach
!pip install cachetools
!pip install certifi
!pip install cffi
!pip install cloudpickle
!pip install cryptography
!pip install cycler
!pip install cytoolz
!pip install dask
!pip install decorator
!pip install earthengine-api
!pip install gast
!pip install GDAL
!pip install google-api-python-client
!pip install google-auth
!pip install google-auth-httplib2
!pip install grpcio
!pip install h5py
!pip install html5lib
!pip install httplib2
!pip install idna
!pip install imageio
!pip install ipython
!pip install ipython-genutils
!pip install jedi
!pip install joblib
!pip install Keras-Applications
!pip install Keras-Preprocessing
!pip install kiwisolver
!pip install Markdown
!pip install matplotlib
!pip install mkl-fft
!pip install mkl-random
!pip install networkx
!pip install numpy
!pip install oauth2client
!pip install olefile
!pip install pandas
!pip install parso
!pip install pexpect
!pip install pickleshare
!pip install Pillow
!pip install prompt-toolkit
!pip install protobuf
!pip install ptyprocess
!pip install pyasn1
!pip install pyasn1-modules
!pip install pycparser
!pip install Pygments
!pip install pyOpenSSL
!pip install pyparsing
!pip install python-dateutil
!pip install pytz
!pip install PyWavelets
!pip install rsa
!pip install scikit-image
!pip install scikit-learn
!pip install scipy
!pip install six
!pip install soupsieve
!pip install tensorboard
!pip install tensorflow
!pip install tensorflow-tensorboard
!pip install termcolor
!pip install toolz
!pip install tornado
!pip install traitlets
!pip install uritemplate
!pip install wcwidth
!pip install Werkzeug
!pip install xarray

     |████████████████████████████████| 112kB 7.6MB/s 
     |████████████████████████████████| 3.2MB 8.8MB/s 
     |████████████████████████████████| 481kB 7.2MB/s 
  Created wheel for cytoolz: filename=cytoolz-0.11.0-cp37-cp37m-linux_x86_64.whl size=1224595 sha256=0fa4b180d1b2ca047ebf600f86a103b13ab8ac63d968abdbb39c6d17f5a26c9a
  Stored in directory: /root/.cache/pip/wheels/a1/32/3c/9c9926b510647cacdde744b2c7acdf1ccd5896fbb7f8d5df0c
Successfully built cytoolz
     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 245kB 8.7MB/s 
     |████████████████████████████████| 3.5MB 37.1MB/s 
     |████████████████████████████████| 32.8MB 93kB/s 
ERROR: Could not find a version that satisfies the requirement intel-opencl-rt==2021.3.0 (from dpcpp_cpp_rt->mkl-fft) (from versions: none)
ERROR: No matching distribution found for intel-opencl-rt==2021.3.0 (from dpcpp_cpp_rt->mkl-fft)
     |████████████████████████████████| 389kB 6.9MB/s 
  Using cached https://

In [ ]:
import ee
import time
import sys
import numpy as np
import pandas as pd
import itertools
import os
import urllib.request, urllib.parse, urllib.error

In [ ]:
import ee
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=7PW4OFtrAgC1oTvlNOOE0fI3TtDa57zFGtlzxSm1MLQ&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWilr9eqaBPKeY0JoZ4faJsSjLevXsWkVpx_cO8YApyvd8LBuOF3nWs

Successfully saved authorization token.


In [ ]:
import ee
ee.Initialize()

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
def export_oneimage(img,folder,name,region,scale,crs):
  task = ee.batch.Export.image(img, name, {
      'driveFolder':folder, # folder where image will be saved
      'driveFileNamePrefix':name, # prefix to export image
      'region': region, # Region to be extract
      'scale':scale, # Howmuch image can zoom
      'crs':crs  #convert google earth image into coordinate system x-y in remote sensing
  })
  task.start() # task start process of extract the image from google earth
  while task.status()['state'] == 'RUNNING':
    print('Running...')
    # Perhaps task.cancel() at some point.
    time.sleep(10)
  print('Done.', task.status())

In [ ]:
# data that provide coordinate for download images
locations = pd.read_csv('/content/drive/MyDrive/Crop_Yield/locations_final.csv')

In [ ]:
def appendBand(current, previous):
    # Rename the band
    #previous=ee.Image(previous)
    #current = current.select([0,1,2,3])
    # Append it to the result (Note: only return current item on first element/iteration)
    #accum = ee.Algorithms.If(ee.Algorithms.IsEqual(previous,None), current, previous.addBands(ee.Image(current),overwrite=True))

    print("Before accum")
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(ee.Image(previous),None), current.select([0]), ee.Image(previous).addBands(ee.Image(current.select([0]))))
    #accum = ee.Image(previous).addBands(current.select([0]))
    # Return the accumulation
    return accum

In [ ]:
county_region = ee.FeatureCollection('ft:18Ayj5e7JxxtTPm1BdMnnzWbZMrxMB49eqGDTsaSp')

In [ ]:
imgcoll = ee.ImageCollection('MODIS/006/MCD12Q1').filterBounds(ee.Geometry.Rectangle(-106.5, 50,-64, 23)).filterDate('2001-12-31','2015-12-31')
#imgcoll = ee.ImageCollection('MODIS/006/MCD12Q1').filterBounds(ee.Geometry.Rectangle(-106.5, 50,-64, 23))
count = imgcoll.size()
print('Count: ', str(count.getInfo())+'\n')
#img=imgcoll.iterate(appendBand)
#mg2 = ee.Image(img);
first = imgcoll.first()
img = ee.Image(imgcoll.iterate(appendBand))
img=ee.Image(img)

Count:  14

Before accum
Before accum


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
for loc1, loc2, lat, lon in locations[:].values:
    fname = '{}_{}'.format(int(loc1), int(loc2))

    offset = 0.11
    scale  = 500
    crs='EPSG:4326'

    region = str([
        [lat - offset, lon + offset],
        [lat + offset, lon + offset],
        [lat + offset, lon - offset],
        [lat - offset, lon - offset]])

    #try:
    export_oneimage(img,'Final1_Data_mask_Modis_Land_Cover',fname,region,scale,crs)
    #except Exception as e:
    #  print('retry',e)
    #break

Done. {'state': 'READY', 'description': '1_65', 'creation_timestamp_ms': 1625040466279, 'update_timestamp_ms': 1625040466279, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'FMGJLFBAV43QHX7PHG4NRU5S', 'name': 'projects/earthengine-legacy/operations/FMGJLFBAV43QHX7PHG4NRU5S'}
Done. {'state': 'READY', 'description': '1_85', 'creation_timestamp_ms': 1625040466983, 'update_timestamp_ms': 1625040466983, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'HRMIEU3YHMRJ5O3AH3J3GY74', 'name': 'projects/earthengine-legacy/operations/HRMIEU3YHMRJ5O3AH3J3GY74'}
Done. {'state': 'READY', 'description': '1_105', 'creation_timestamp_ms': 1625040467970, 'update_timestamp_ms': 1625040467970, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'GWP4WLZROVEY2BYS5L6Q3J2H', 'name': 'projects/earthengine-legacy/operations/GWP4WLZROVEY2BYS5L6Q3J2H'}
Done. {'state': 'READY', 'description': '1_119', 'creation_timestamp_ms': 1625040468677, 'update_timestamp_ms': 1625040468677, 'star